# Exploring Classification Methods with Pokemon

Classification Alogirthims

- rsteo
- arst
- rast
- rst
- ast


### Import Preliminaries

In [3]:
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
#from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import KFold

from keras import models
from keras import layers
from keras import optimizers
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import GaussianNB
warnings.filterwarnings('ignore')

Using TensorFlow backend.


ImportError: cannot import name 'pywrap_tensorflow'


### Import Data


In [ ]:
df = pd.read_csv('Data/pokemon.csv')
df.info()

# Cleaning the Data

In [ ]:
print(df.shape)
cdf = df[df['Legendary'] == False]
cdf[['Type 1', 'Type 2']] = cdf[['Type 1', 'Type 2']].fillna(value='')
cdf['Type'] = cdf.loc[:,'Type 1'] + " " + cdf.loc[:,'Type 2']
cdf = cdf.drop(['Legendary','Name','#','Type 1','Type 2'], axis=1)
cdf['Type'] =  cdf['Type'].astype('category').cat.codes
#cdf['Type 2'] =  cdf['Type 2'].astype('category').cat.codes
print(cdf.shape)

In [ ]:
train_y = cdf.Type.values
train_x = cdf.drop(['Type'], axis=1).values

### Decision Tree

In [ ]:

model = DecisionTreeClassifier()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)


crossvalidation = KFold(n_splits=10, random_state=1)
scores = cross_val_score(model, train_x, train_y, scoring ='accuracy', 
                         cv=crossvalidation, n_jobs =1)

print('Folds: %i, accuracy: %.4f std: %.2f' % 
      (len(scores), np.mean(np.abs(scores)), np.std(scores)))

### Random Forest

In [ ]:

model = RandomForestClassifier()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)

crossvalidation = KFold(n_splits=10, random_state=1)
scores = cross_val_score(model, train_x, train_y, scoring ='accuracy', 
                         cv=crossvalidation, n_jobs =1)

print('Folds: %i, accuracy: %.4f std: %.2f' % 
      (len(scores), np.mean(np.abs(scores)), np.std(scores)))

In [ ]:

model = GaussianNB()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)

crossvalidation = KFold(n_splits=10, random_state=1)
scores = cross_val_score(model, train_x, train_y, scoring ='accuracy', 
                         cv=crossvalidation, n_jobs =1)

print('Folds: %i, accuracy: %.8f std: %.2f' % 
      (len(scores), np.mean(np.abs(scores)), np.std(scores)))

### CNN Classifier

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
  
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, labels] = 1
    return results

In [ ]:
from keras.utils.np_utils import to_categorical
#train_y = vectorize_sequences(train_y)
train_y = to_categorical(train_y)

In [ ]:
train_x.shape

In [ ]:
train_y.shape

In [ ]:
train_x.shape[0]

In [ ]:
model = models.Sequential()
model.add(layers.Dense(4, activation='relu',input_shape=(train_x.shape[1],)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(train_y.shape[1], activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(train_x, train_y, epochs=20, batch_size=10, verbose=0)

In [ ]:
model.summary()